In [1]:
import os
import kagglehub
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import classification_report
from tqdm import tqdm

In [2]:
path = kagglehub.dataset_download("ankit1743/skyview-an-aerial-landscape-dataset")
print("Path to dataset files:", path)

DATA_DIR = os.path.join(path, "Aerial_Landscapes")
BATCH_SIZE = 16
NUM_CLASSES = 15
EPOCHS = 10
DEVICE = torch.device("cuda")

Path to dataset files: /kaggle/input/skyview-an-aerial-landscape-dataset


In [3]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
])
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [4]:
def get_model():
    model = models.resnet50(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
    return model.to(DEVICE)

In [5]:
def train_model(model, train_loader, test_loader):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(model(imgs), labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Loss: {total_loss / len(train_loader):.4f}")
        evaluate_model(model, test_loader)

In [6]:
def evaluate_model(model, loader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(DEVICE)
            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    print(classification_report(y_true, y_pred, zero_division=0))

In [7]:
def main():
    dataset = datasets.ImageFolder(DATA_DIR, transform=train_transforms)
    train_len = int(0.8 * len(dataset))
    test_len = len(dataset) - train_len
    train_data, test_data = random_split(dataset, [train_len, test_len])

    train_data.dataset.transform = train_transforms
    test_data.dataset.transform = test_transforms

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

    model = get_model()
    train_model(model, train_loader, test_loader)

if __name__ == '__main__':
    main()


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 189MB/s]
Epoch 1/10: 100%|██████████| 600/600 [02:36<00:00,  3.83it/s]


Loss: 0.4578
              precision    recall  f1-score   support

           0       0.99      0.95      0.97       167
           1       0.98      0.93      0.95       163
           2       0.94      0.99      0.97       156
           3       0.93      0.99      0.96       156
           4       0.91      0.99      0.94       137
           5       0.97      0.99      0.98       183
           6       0.99      0.95      0.97       173
           7       0.88      0.97      0.92       135
           8       0.98      0.96      0.97       151
           9       0.98      0.94      0.96       156
          10       0.99      0.97      0.98       172
          11       0.99      0.97      0.98       159
          12       0.96      0.92      0.94       149
          13       0.98      0.97      0.98       172
          14       0.98      0.95      0.96       171

    accuracy                           0.96      2400
   macro avg       0.96      0.96      0.96      2400
weighted avg 

Epoch 2/10: 100%|██████████| 600/600 [01:21<00:00,  7.38it/s]


Loss: 0.1813
              precision    recall  f1-score   support

           0       0.94      0.99      0.96       167
           1       0.96      0.93      0.94       163
           2       0.96      0.96      0.96       156
           3       0.93      0.98      0.96       156
           4       0.92      0.99      0.95       137
           5       0.95      0.97      0.96       183
           6       0.95      0.94      0.94       173
           7       0.96      0.97      0.96       135
           8       0.97      0.94      0.95       151
           9       0.96      0.96      0.96       156
          10       0.99      0.98      0.99       172
          11       0.99      0.94      0.97       159
          12       0.98      0.95      0.97       149
          13       0.99      0.97      0.98       172
          14       0.97      0.94      0.95       171

    accuracy                           0.96      2400
   macro avg       0.96      0.96      0.96      2400
weighted avg 

Epoch 3/10: 100%|██████████| 600/600 [01:21<00:00,  7.34it/s]


...

Loss: 0.0594
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       167
           1       0.97      0.91      0.94       163
           2       0.99      0.99      0.99       156
           3       0.91      0.98      0.94       156
           4       0.96      0.95      0.95       137
           5       0.97      0.99      0.98       183
           6       0.98      0.98      0.98       173
           7       0.97      0.93      0.95       135
           8       0.99      0.96      0.98       151
           9       0.93      0.97      0.95       156
          10       0.99      0.97      0.98       172
          11       0.99      0.97      0.98       159
          12       0.92      0.99      0.95       149
          13       0.98      0.94      0.96       172
          14       0.95      0.93      0.94       171

    accuracy                           0.96      2400
   macro avg       0.96      0.96      0.96      2400
weighted avg 

Epoch 9/10: 100%|██████████| 600/600 [01:21<00:00,  7.40it/s]


Loss: 0.0630
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       167
           1       0.97      0.94      0.95       163
           2       0.93      0.97      0.95       156
           3       0.93      0.98      0.96       156
           4       0.85      1.00      0.92       137
           5       0.96      0.99      0.97       183
           6       0.98      0.98      0.98       173
           7       0.98      0.94      0.96       135
           8       1.00      0.93      0.96       151
           9       0.98      0.82      0.90       156
          10       0.99      0.99      0.99       172
          11       0.99      0.97      0.98       159
          12       0.92      0.93      0.93       149
          13       0.98      0.98      0.98       172
          14       0.94      0.95      0.94       171

    accuracy                           0.96      2400
   macro avg       0.96      0.96      0.96      2400
weighted avg 

Epoch 10/10: 100%|██████████| 600/600 [01:20<00:00,  7.42it/s]


Loss: 0.0505
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       167
           1       0.99      0.84      0.91       163
           2       0.98      0.96      0.97       156
           3       0.93      0.97      0.95       156
           4       0.91      0.98      0.94       137
           5       0.98      0.99      0.99       183
           6       0.99      0.95      0.97       173
           7       0.99      0.94      0.97       135
           8       1.00      0.92      0.96       151
           9       0.97      0.96      0.96       156
          10       0.95      0.99      0.97       172
          11       0.99      0.97      0.98       159
          12       0.92      0.97      0.94       149
          13       0.97      0.99      0.98       172
          14       0.88      0.99      0.93       171

    accuracy                           0.96      2400
   macro avg       0.96      0.96      0.96      2400
weighted avg 